In [1]:
import urllib
import json
import requests
import time
headers = {'Content-Type': 'multipart/form-data'}

In [3]:
def read_json(filename):
    with open(filename) as f:
        return json.load(f)
    
def write_json(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        
def get_tgt_langs(filename):
    data = read_json(filename)
    tgt_langs = list()
    for q in data['questions']:
        for l in q['question']:
            if l['language'] not in tgt_langs:
                tgt_langs.append(l['language'])
        break
    return tgt_langs

In [4]:
url_postfix = """experimentData={{"type":"QA","matching":"STRONG_ENTITY_MATCH","annotator":["{annotator}"],"dataset":["{dataset}"],"answerFiles":[],"questionLanguage":"{lang}"}}"""

dataset_dict = {
    #'English': 'en',
    #'German': 'de',
    #'Russian': 'ru',
    'Bashkir': 'ba',
    'Lithuanian': 'lt',
    'Ukrainian': 'uk',
    'Belarusian': 'be',
    #'Armenian': 'arm',
    'French': 'fr'
}

annotators = [
    'QAnswer',
    'DeepPavlov',
    'Platypus'
]

In [5]:
for annotator_ in annotators:
    annotator = "NIFWS_{0}(https://webengineering.ins.hs-anhalt.de:41021/{1}/gerbil_wikidata)".format(annotator_, annotator_.lower())
    for src_lang, code in zip(dataset_dict.keys(), dataset_dict.values()):
        dataset_name = "NIFDS_QALD++Test+{0}+HelsinkiNLP(qald_test_wikidata-helsinki-{1}.json)".format(src_lang, code)
        for lang in get_tgt_langs(f"../data/translated/qald_test_wikidata-yandex-{code}.json"):
            if annotator_ == 'DeepPavlov' and (lang != 'ru' and lang != 'en'):
                continue
            if annotator_ == 'Platypus' and (lang != 'fr' and lang != 'en'):
                continue
                
            execute_url = "http://gerbil-qa.aksw.org/gerbil/execute?" + url_postfix.format(
                annotator=annotator,
                dataset=dataset_name,
                lang=lang
            )
            is_ok = False
            while not is_ok:
                try:
                    if requests.get("http://gerbil-qa.aksw.org/gerbil/running").text == '':
                        response = requests.get(execute_url)
                        print(annotator, src_lang, lang, "http://gerbil-qa.aksw.org/gerbil/experiment?id={0}".format(response.text))
                        is_ok = True
                    else:
                        time.sleep(60)
                except:
                    time.sleep(60)

NIFWS_QAnswer(https://webengineering.ins.hs-anhalt.de:41021/qanswer/gerbil_wikidata) Bashkir en http://gerbil-qa.aksw.org/gerbil/experiment?id=202111150035
NIFWS_QAnswer(https://webengineering.ins.hs-anhalt.de:41021/qanswer/gerbil_wikidata) Bashkir de http://gerbil-qa.aksw.org/gerbil/experiment?id=202111150036
NIFWS_QAnswer(https://webengineering.ins.hs-anhalt.de:41021/qanswer/gerbil_wikidata) Bashkir ru http://gerbil-qa.aksw.org/gerbil/experiment?id=202111150037
NIFWS_QAnswer(https://webengineering.ins.hs-anhalt.de:41021/qanswer/gerbil_wikidata) Bashkir fr http://gerbil-qa.aksw.org/gerbil/experiment?id=202111150038
NIFWS_QAnswer(https://webengineering.ins.hs-anhalt.de:41021/qanswer/gerbil_wikidata) Lithuanian en http://gerbil-qa.aksw.org/gerbil/experiment?id=202111150039
NIFWS_QAnswer(https://webengineering.ins.hs-anhalt.de:41021/qanswer/gerbil_wikidata) Lithuanian de http://gerbil-qa.aksw.org/gerbil/experiment?id=202111150040
NIFWS_QAnswer(https://webengineering.ins.hs-anhalt.de:4102

# Native Language

In [10]:
url_postfix = """experimentData={{"type":"QA","matching":"STRONG_ENTITY_MATCH","annotator":["{annotator}"],"dataset":["{dataset}"],"answerFiles":[],"questionLanguage":"{lang}"}}"""

native_langs = ['de', 'ru', 'fr'] #['en', 'de', 'ru', 'fr']
annotators = ['QAnswer'] # , 'DeepPavlov', 'Platypus']
kb = 'DBpedia'

In [11]:
# QALDNine Test Wikidata
for split in ['Test', 'Train']:
    for annotator_ in annotators:
        annotator = "NIFWS_{0}(https://webengineering.ins.hs-anhalt.de:41021/{1}/gerbil_{2})".format(annotator_, annotator_.lower(), kb.lower())
        for lang in native_langs:
            if annotator_ == 'DeepPavlov' and (lang != 'ru' and lang != 'en'):
                continue
            if annotator_ == 'Platypus' and (lang != 'fr' and lang != 'en'):
                continue

            dataset_name = f"NIFDS_QALDNine+{split}+{kb}(qald_{split.lower()}_{kb.lower()}.json)"
            execute_url = "http://gerbil-qa.aksw.org/gerbil/execute?" + url_postfix.format(
                annotator=annotator,
                dataset=dataset_name,
                lang=lang
            )
            is_ok = False
            while not is_ok:
                try:
                    if 'QALDPlus' not in requests.get("http://gerbil-qa.aksw.org/gerbil/running").text:
                        print(dataset_name)
                        response = requests.get(execute_url)
                        print(annotator, lang, "http://gerbil-qa.aksw.org/gerbil/experiment?id={0}".format(response.text))
                        is_ok = True
                    else:
                        print("sleep")
                        time.sleep(60)
                except Exception as e:
                    print(str(e))
                    time.sleep(60)

NIFDS_QALDNine+Test+DBpedia(qald_test_dbpedia.json)
NIFWS_QAnswer(https://webengineering.ins.hs-anhalt.de:41021/qanswer/gerbil_dbpedia) de http://gerbil-qa.aksw.org/gerbil/experiment?id=202112190016
NIFDS_QALDNine+Test+DBpedia(qald_test_dbpedia.json)
NIFWS_QAnswer(https://webengineering.ins.hs-anhalt.de:41021/qanswer/gerbil_dbpedia) ru http://gerbil-qa.aksw.org/gerbil/experiment?id=202112190017
NIFDS_QALDNine+Test+DBpedia(qald_test_dbpedia.json)
NIFWS_QAnswer(https://webengineering.ins.hs-anhalt.de:41021/qanswer/gerbil_dbpedia) fr http://gerbil-qa.aksw.org/gerbil/experiment?id=202112190018
NIFDS_QALDNine+Train+DBpedia(qald_train_dbpedia.json)
NIFWS_QAnswer(https://webengineering.ins.hs-anhalt.de:41021/qanswer/gerbil_dbpedia) de http://gerbil-qa.aksw.org/gerbil/experiment?id=202112190019
NIFDS_QALDNine+Train+DBpedia(qald_train_dbpedia.json)
NIFWS_QAnswer(https://webengineering.ins.hs-anhalt.de:41021/qanswer/gerbil_dbpedia) ru http://gerbil-qa.aksw.org/gerbil/experiment?id=202112190020
N